# txt 크롤링

In [17]:
# 1. 필요한 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import time
import os
from typing import Optional, List, Dict, Any
from tqdm import tqdm
import re
from lxml import html

# 2. HTML 파싱 관련 함수들
def parse_book_title(soup: BeautifulSoup) -> str:
    """책 제목을 파싱"""
    return soup.select_one('h1').text.strip()

def parse_toc_links(soup: BeautifulSoup) -> List[Dict[str, str]]:
    """목차 링크들을 파싱"""
    links = soup.select('div.toc a')
    processed_links = []
    
    for link in links:
        href = link.get('href', '')
        # JavaScript 함수에서 페이지 번호 추출
        if href.startswith('javascript:'):
            page_num = re.search(r'page\((\d+)\)', href)
            if page_num:
                href = f"/{page_num.group(1)}"
        
        processed_links.append({
            'text': link.text.strip(),
            'href': href
        })
    
    return processed_links

def parse_chapter_content(response: requests.Response) -> str:
    """챕터 내용을 파싱 (XPath 사용)"""
    # HTML 파싱
    tree = html.fromstring(response.content)
    
    # XPath로 본문 내용 추출
    main_content = tree.xpath('/html/body/div/div[1]/div[2]/div[5]')
    
    if not main_content:
        return ""
    
    # 본문 내용에서 불필요한 요소 제거
    for element in main_content[0].xpath('.//script | .//style | .//iframe | .//noscript'):
        element.getparent().remove(element)
    
    # 텍스트 추출 및 정리
    text_content = main_content[0].text_content()
    
    # 줄바꿈 정리
    lines = [line.strip() for line in text_content.split('\n') if line.strip()]
    
    # 코드 블록 보존
    code_blocks = main_content[0].xpath('.//pre')
    for code in code_blocks:
        code_text = code.text_content()
        if code_text:
            # 코드 블록을 ```로 감싸기
            code_text = f"```\n{code_text}\n```"
            # 원본 코드 블록을 변환된 텍스트로 교체
            code.getparent().text = code_text
    
    # 이미지 처리
    images = main_content[0].xpath('.//img')
    for img in images:
        alt_text = img.get('alt', '')
        img_text = f"[이미지: {alt_text}]"
        img.getparent().text = img_text
    
    # 링크 처리
    links = main_content[0].xpath('.//a')
    for link in links:
        href = link.get('href', '')
        text = link.text_content().strip()
        if href:
            link_text = f"{text} ({href})"
            link.getparent().text = link_text
    
    # HTML 태그 제거 후 텍스트만 추출
    text_content = main_content[0].text_content()
    
    # 줄바꿈 정리 및 불필요한 공백 제거
    lines = []
    for line in text_content.split('\n'):
        line = line.strip()
        if line and not line.isspace():
            lines.append(line)
    
    return '\n'.join(lines)

# 3. HTTP 요청 관련 함수들
def create_session() -> requests.Session:
    """세션 생성"""
    return requests.Session()

def get_page_content(session: requests.Session, url: str) -> requests.Response:
    """페이지 내용을 가져옴"""
    return session.get(url)

# 4. 책 데이터 수집 함수들
def collect_chapter_data(session: requests.Session, chapter_link: Dict[str, str], pbar: tqdm) -> Dict[str, str]:
    """개별 챕터 데이터 수집"""
    chapter_url = f"https://wikidocs.net{chapter_link['href']}"
    response = get_page_content(session, chapter_url)
    
    # 진행 상태 업데이트
    pbar.set_description(f"크롤링 중: {chapter_link['text'][:30]}...")
    pbar.update(1)
    
    return {
        'title': chapter_link['text'],
        'content': parse_chapter_content(response),
        'url': chapter_url
    }

def collect_book_data(url: str) -> Dict[str, Any]:
    """전체 책 데이터 수집"""
    session = create_session()
    response = get_page_content(session, url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 책 제목과 목차 링크 수집
    book_title = parse_book_title(soup)
    toc_links = parse_toc_links(soup)
    
    # tqdm으로 진행 상태 표시
    with tqdm(total=len(toc_links), desc="챕터 크롤링", unit="챕터") as pbar:
        # 각 챕터 데이터 수집
        chapters = []
        for link in toc_links:
            chapter = collect_chapter_data(session, link, pbar)
            chapters.append(chapter)
            time.sleep(1)  # 서버 부하 방지
    
    return {
        'title': book_title,
        'chapters': chapters,
        'url': url
    }

# 5. 파일 저장 관련 함수들
def format_book_content(book: Dict[str, Any]) -> str:
    """책 내용을 텍스트 형식으로 변환"""
    content = [f"=== {book['title']} ===\n"]
    
    for chapter in book['chapters']:
        content.extend([
            f"\n--- {chapter['title']} ---\n",
            chapter['content'],
            "\n" + "="*50 + "\n"  # 챕터 구분선
        ])
    
    return '\n'.join(content)

def save_book_to_file(content: str, url: str, save_dir: Optional[str] = None) -> str:
    """책 내용을 파일로 저장"""
    if save_dir is None:
        save_dir = os.getcwd()
    
    book_number = url.split('/')[-1]
    filename = f"wikidocs_book_{book_number}.txt"
    filepath = os.path.join(save_dir, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    
    return filepath

# 6. 메인 크롤링 함수
def crawl_wikidocs_book(url: str, save_dir: Optional[str] = None) -> Optional[str]:
    """
    위키독스 책의 모든 내용을 크롤링하여 텍스트 파일로 저장
    
    Args:
        url (str): 위키독스 책 URL
        save_dir (str, optional): 저장할 디렉토리 경로
    
    Returns:
        Optional[str]: 저장된 파일 경로 또는 에러 발생 시 None
    """
    try:
        print(f"책 페이지 접속 중: {url}")
        book = collect_book_data(url)
        print(f"책 제목: {book['title']}")
        print(f"총 {len(book['chapters'])}개의 챕터 발견")
        
        print("파일 저장 중...")
        content = format_book_content(book)
        filepath = save_book_to_file(content, url, save_dir)
        
        print(f"\n저장 완료: {filepath}")
        return filepath
        
    except Exception as e:
        print(f"에러 발생: {str(e)}")
        return None

# pdf 크롤링

In [32]:
# import requests
# from bs4 import BeautifulSoup
# from lxml import html
# from reportlab.lib import colors
# from reportlab.lib.pagesizes import A4
# from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
# from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
# from reportlab.pdfbase import pdfmetrics
# from reportlab.pdfbase.ttfonts import TTFont
# import os
# import time
# import re
# from typing import Dict, List, Optional, Any
# from tqdm import tqdm
# import io

# def clean_html_text(text: str) -> str:
#     """HTML 태그 제거 및 텍스트 정리"""
#     # HTML 태그 제거
#     text = re.sub(r'<[^>]+>', '', text)
    
#     # HTML 엔티티 처리
#     text = text.replace('&nbsp;', ' ')
#     text = text.replace('&lt;', '<')
#     text = text.replace('&gt;', '>')
#     text = text.replace('&amp;', '&')
#     text = text.replace('&quot;', '"')
#     text = text.replace('&apos;', "'")
    
#     # 불필요한 공백 제거
#     text = re.sub(r'\s+', ' ', text)
#     text = text.strip()
    
#     return text

# def download_image(session: requests.Session, img_url: str) -> Optional[bytes]:
#     """이미지 다운로드"""
#     try:
#         response = session.get(img_url)
#         if response.status_code == 200:
#             return response.content
#     except Exception as e:
#         print(f"이미지 다운로드 실패: {str(e)}")
#     return None

# def parse_book_title(response: requests.Response) -> str:
#     """책 제목 파싱"""
#     soup = BeautifulSoup(response.text, 'html.parser')
#     title = soup.find('h1', class_='book_title')
#     return title.text.strip() if title else "제목 없음"

# def parse_toc_links(response: requests.Response) -> List[Dict[str, str]]:
#     """목차 링크 파싱"""
#     soup = BeautifulSoup(response.text, 'html.parser')
#     toc = soup.find('div', class_='toc')
#     if not toc:
#         return []
    
#     links = []
#     for link in toc.find_all('a'):
#         href = link.get('href', '')
#         # JavaScript URL 처리
#         if href.startswith('javascript:'):
#             # page(숫자) 형식에서 숫자 추출
#             page_num = re.search(r'page\((\d+)\)', href)
#             if page_num:
#                 href = f"/page/{page_num.group(1)}"
        
#         links.append({
#             'text': link.text.strip(),
#             'href': href
#         })
#     return links

# def parse_chapter_content(response: requests.Response, session: requests.Session) -> Dict[str, Any]:
#     """챕터 내용을 파싱 (XPath 사용)"""
#     tree = html.fromstring(response.content)
#     main_content = tree.xpath('/html/body/div/div[1]/div[2]/div[5]')
    
#     if not main_content:
#         return {"text": "", "images": []}
    
#     # 불필요한 요소 제거
#     for element in main_content[0].xpath('.//script | .//style | .//iframe | .//noscript'):
#         element.getparent().remove(element)
    
#     # 텍스트 추출 및 정리
#     text_content = main_content[0].text_content()
#     cleaned_text = clean_html_text(text_content)
#     lines = [line.strip() for line in cleaned_text.split('\n') if line.strip()]
    
#     # 이미지 처리
#     images = []
#     img_elements = main_content[0].xpath('.//img')
#     for img in img_elements:
#         img_url = img.get('src', '')
#         if img_url:
#             if not img_url.startswith('http'):
#                 img_url = f"https://wikidocs.net{img_url}"
            
#             img_data = download_image(session, img_url)
#             if img_data:
#                 alt_text = img.get('alt', '')
#                 images.append({
#                     'data': img_data,
#                     'alt': alt_text
#                 })
    
#     # 코드 블록 보존
#     code_blocks = main_content[0].xpath('.//pre')
#     for code in code_blocks:
#         code_text = code.text_content()
#         if code_text:
#             code_text = f"```\n{code_text}\n```"
#             code.getparent().text = code_text
    
#     # 링크 처리
#     links = main_content[0].xpath('.//a')
#     for link in links:
#         href = link.get('href', '')
#         text = link.text_content().strip()
#         if href:
#             link_text = f"{text} ({href})"
#             link.getparent().text = link_text
    
#     return {
#         "text": '\n'.join(lines),
#         "images": images
#     }

# def register_fonts() -> bool:
#     """한글 폰트 등록"""
#     try:
#         # Windows의 경우
#         if os.name == 'nt':
#             pdfmetrics.registerFont(TTFont('MalgunGothic', 'C:/Windows/Fonts/malgun.ttf'))
#             pdfmetrics.registerFont(TTFont('MalgunGothicBold', 'C:/Windows/Fonts/malgunbd.ttf'))
#             return True
#         # macOS의 경우
#         elif os.name == 'posix':
#             pdfmetrics.registerFont(TTFont('AppleGothic', '/System/Library/Fonts/AppleGothic.ttf'))
#             return True
#     except Exception as e:
#         print(f"폰트 등록 실패: {str(e)}")
#     return False

# def create_pdf_styles() -> Dict[str, ParagraphStyle]:
#     """PDF 스타일 생성"""
#     styles = getSampleStyleSheet()
    
#     # 한글 폰트 사용 가능 여부 확인
#     has_korean_font = register_fonts()
#     default_font = 'MalgunGothic' if has_korean_font else 'Helvetica'
#     default_bold_font = 'MalgunGothicBold' if has_korean_font else 'Helvetica-Bold'
    
#     # 책 제목 스타일
#     styles.add(ParagraphStyle(
#         name='BookTitle',
#         parent=styles['Heading1'],
#         fontSize=24,
#         spaceAfter=30,
#         fontName=default_bold_font
#     ))
    
#     # 챕터 제목 스타일
#     styles.add(ParagraphStyle(
#         name='WikiChapterTitle',
#         parent=styles['Heading2'],
#         fontSize=18,
#         spaceAfter=20,
#         fontName=default_bold_font
#     ))
    
#     # 본문 스타일
#     styles.add(ParagraphStyle(
#         name='WikiBodyText',
#         parent=styles['Normal'],
#         fontSize=12,
#         leading=18,
#         fontName=default_font
#     ))
    
#     # 코드 블록 스타일
#     styles.add(ParagraphStyle(
#         name='WikiCodeBlock',
#         parent=styles['Normal'],
#         fontSize=10,
#         leading=14,
#         fontName='Courier',
#         textColor=colors.darkblue,
#         backColor=colors.lightgrey,
#         borderWidth=1,
#         borderColor=colors.grey,
#         borderPadding=5
#     ))
    
#     return styles

# def create_pdf_content(book: Dict[str, Any], styles) -> List:
#     """PDF 내용 생성"""
#     content = []
    
#     # 책 제목
#     title = clean_html_text(book['title'])
#     content.append(Paragraph(title, styles['BookTitle']))
#     content.append(Spacer(1, 30))
    
#     # 각 챕터
#     for chapter in book['chapters']:
#         # 챕터 제목
#         chapter_title = clean_html_text(chapter['title'])
#         content.append(Paragraph(chapter_title, styles['WikiChapterTitle']))
#         content.append(Spacer(1, 10))
        
#         # 본문 내용
#         paragraphs = chapter['content'].split('\n')
#         for para in paragraphs:
#             if para.strip():
#                 if para.startswith('```'):
#                     # 코드 블록
#                     content.append(Paragraph(para, styles['WikiCodeBlock']))
#                 else:
#                     # 일반 텍스트
#                     cleaned_para = clean_html_text(para)
#                     if cleaned_para:
#                         content.append(Paragraph(cleaned_para, styles['WikiBodyText']))
        
#         # 이미지 추가
#         for img in chapter['images']:
#             try:
#                 # 이미지 데이터를 ReportLab Image 객체로 변환
#                 img_data = io.BytesIO(img['data'])
#                 img_width = 400  # 이미지 너비 설정
#                 img_height = 300  # 이미지 높이 설정
#                 img = Image(img_data, width=img_width, height=img_height)
#                 content.append(img)
#                 content.append(Spacer(1, 10))
                
#                 # 이미지 설명 추가
#                 if img['alt']:
#                     content.append(Paragraph(f"[이미지 설명: {img['alt']}]", styles['WikiBodyText']))
#                     content.append(Spacer(1, 10))
#             except Exception as e:
#                 print(f"이미지 처리 실패: {str(e)}")
        
#         content.append(Spacer(1, 20))
    
#     return content

# def save_to_pdf(book: Dict[str, Any], save_dir: str):
#     """책 내용을 PDF로 저장"""
#     # URL에서 책 번호 추출
#     book_number = book['url'].split('/')[-1]
#     pdf_path = os.path.join(save_dir, f'wikidocs_{book_number}.pdf')
    
#     # PDF 생성
#     doc = SimpleDocTemplate(
#         pdf_path,
#         pagesize=A4,
#         rightMargin=72,
#         leftMargin=72,
#         topMargin=72,
#         bottomMargin=72
#     )
    
#     # 스타일 생성
#     styles = create_pdf_styles()
    
#     # 내용 생성
#     content = create_pdf_content(book, styles)
    
#     # PDF 저장
#     doc.build(content)
#     print(f"PDF 저장 완료: {pdf_path}")

# def create_session() -> requests.Session:
#     """세션 생성"""
#     session = requests.Session()
#     session.headers.update({
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     })
#     return session

# def get_page_content(session: requests.Session, url: str) -> requests.Response:
#     """페이지 내용 가져오기"""
#     try:
#         response = session.get(url)
#         response.raise_for_status()
#         time.sleep(1)  # 서버 부하 방지
#         return response
#     except requests.RequestException as e:
#         print(f"페이지 요청 실패: {str(e)}")
#         raise

# def collect_chapter_data(session: requests.Session, chapter_link: Dict[str, str], pbar: tqdm) -> Dict[str, Any]:
#     """개별 챕터 데이터 수집"""
#     # URL 구성 수정
#     if chapter_link['href'].startswith('/page/'):
#         chapter_url = f"https://wikidocs.net{chapter_link['href']}"
#     else:
#         chapter_url = f"https://wikidocs.net{chapter_link['href']}"
    
#     response = get_page_content(session, chapter_url)
    
#     pbar.set_description(f"크롤링 중: {chapter_link['text'][:30]}...")
#     pbar.update(1)
    
#     content = parse_chapter_content(response, session)
    
#     return {
#         'title': chapter_link['text'],
#         'content': content['text'],
#         'images': content['images'],
#         'url': chapter_url
#     }

# def crawl_wikidocs_book(url: str, save_dir: str):
#     """위키독스 책 크롤링 및 PDF 저장"""
#     try:
#         # 저장 디렉토리 생성
#         os.makedirs(save_dir, exist_ok=True)
        
#         # 세션 생성
#         session = create_session()
        
#         # 메인 페이지 요청
#         response = get_page_content(session, url)
        
#         # 책 정보 수집
#         book = {
#             'title': parse_book_title(response),
#             'url': url,
#             'chapters': []
#         }
        
#         # 목차 링크 수집
#         toc_links = parse_toc_links(response)
        
#         # 진행률 표시
#         with tqdm(total=len(toc_links), desc="챕터 크롤링") as pbar:
#             # 각 챕터 데이터 수집
#             for chapter_link in toc_links:
#                 chapter_data = collect_chapter_data(session, chapter_link, pbar)
#                 book['chapters'].append(chapter_data)
        
#         # PDF 저장
#         save_to_pdf(book, save_dir)
        
#     except Exception as e:
#         print(f"크롤링 실패: {str(e)}")
#         raise

# 크롤링 실행

In [33]:
# 크롤링 결과 저장 경로
save_dir = r"../../data/txt"

# 첫번째 책 크롤링
url1 = "https://wikidocs.net/book/14314"  # 첫 번째 책
filepath = crawl_wikidocs_book(url1, save_dir)

크롤링 중: 01. StreamEvent 타입별 정리...: 100%|██████████| 184/184 [04:24<00:00,  1.44s/it]                            


PDF 저장 완료: ../../data/pdf\wikidocs_14314.pdf


In [18]:
# 크롤링 결과 저장 경로
save_dir = r"../../data/txt"

# 두번째 책 크롤링
url2 = "https://wikidocs.net/book/2155"  # 두 번째 책
filepath2 = crawl_wikidocs_book(url2, save_dir)

책 페이지 접속 중: https://wikidocs.net/book/2155


크롤링 중: 24. 교육 문의...: 100%|██████████| 179/179 [03:24<00:00,  1.14s/챕터]                    ]              

책 제목: 딥 러닝을 이용한 자연어 처리 입문
총 179개의 챕터 발견
파일 저장 중...

저장 완료: ../../data/txt\wikidocs_book_2155.txt


In [19]:
# 크롤링 결과 저장 경로
save_dir = r"../../data/txt"

# 세번째 책 크롤링
url3 = "https://wikidocs.net/book/2788"  # 세 번째 책
filepath3 = crawl_wikidocs_book(url3, save_dir)

책 페이지 접속 중: https://wikidocs.net/book/2788


크롤링 중: 구버전) IMDB 리뷰 감성 분류하기(IMDB Movi...: 100%|██████████| 132/132 [02:27<00:00,  1.12s/챕터]          

책 제목: 딥 러닝 파이토치 교과서 - 입문부터 파인튜닝까지
총 132개의 챕터 발견
파일 저장 중...

저장 완료: ../../data/txt\wikidocs_book_2788.txt
